# Api Keys

Many APIs require an API key to authenicate a registered user when making a web request.

This example uses the Guardian Newspaper API.  You will need to register for a developer key first:

https://open-platform.theguardian.com/access/

This example shows how to pass the api key as a query parameter in the URL.

This example also shows how to deal with multiple pages of results coming back from an API.


In [1]:
# Import the libraries
import requests
import urllib
import json
import pandas as pd

Note that I keep my API keys in a file called secret.py.  These keys are my own private keys, so I don't want to share them!

In [2]:
from secret import *

This function builds the query URL, including passing in the API key and the page number requested:

In [3]:
# Function to put together the API query
def getRequestUrl(pageNo):
    url = "https://content.guardianapis.com/search?"
    params = {"q":          "olympic+games",
              "page-size":  50,
              "page":       str(pageNo),
              "from-date":  "2021-05-26",
              "api-key":    guardian_api_key}
    request = url + urllib.parse.urlencode(params)
    return request

In [4]:
# Function to call the API with the given query URL
def getApiData(requestUrl):
    # Call the api
    response = requests.get(requestUrl)
    
    # Extract the text from the response
    data = json.loads(response.text)
    
    # Get the actual data (as specfied in the guardian api)
    results = data["response"]["results"]
    
    # Get other information (as specfied in the guardian api)
    status = data["response"]["status"]
    pages = int(data["response"]["pages"])
    pageSize = int(data["response"]["pageSize"])
    total = int(data["response"]["total"])
    
    print("Response: status {}, pages {}, pageSize {}, total {}".format(status, pages, pageSize, total))
    
    return pages, results


This example just gets the first page of results

In [5]:
# Code to get the first page of data

# Get the first page
requestUrl = getRequestUrl(pageNo=1)
numPages, results = getApiData(requestUrl)

print("Found", numPages, "pages")

df = pd.DataFrame(results)
df.head()

Response: status ok, pages 4, pageSize 50, total 159
Found 4 pages


,id,type,sectionId,sectionName,webPublicationDate,webTitle,webUrl,apiUrl,isHosted,pillarId,pillarName
0,world/2021/jun/04/tokyo-cornered-into-going-ah...,article,world,World news,2021-06-04T10:18:30Z,"Tokyo cornered into going ahead with Games, sa...",https://www.theguardian.com/world/2021/jun/04/...,https://content.guardianapis.com/world/2021/ju...,False,pillar/news,News
1,sport/2021/jun/07/sally-fitzgibbons-continues-...,article,sport,Sport,2021-06-07T05:26:47Z,Sally Fitzgibbons continues hot pre-Olympic fo...,https://www.theguardian.com/sport/2021/jun/07/...,https://content.guardianapis.com/sport/2021/ju...,False,pillar/sport,Sport
2,sport/2021/jun/02/olympics-over-a-barrel-grim-...,article,sport,Sport,2021-06-02T15:15:47Z,IOC holds Tokyo as Olympic hostage to fortune ...,https://www.theguardian.com/sport/2021/jun/02/...,https://content.guardianapis.com/sport/2021/ju...,False,pillar/sport,Sport
3,world/2021/jun/03/olympics-will-go-ahead-says-...,article,world,World news,2021-06-03T05:11:03Z,Olympics chief says Games will proceed after C...,https://www.theguardian.com/world/2021/jun/03/...,https://content.guardianapis.com/world/2021/ju...,False,pillar/news,News
4,world/2021/may/26/tokyo-olympics-asahi-shimbun...,article,world,World news,2021-05-26T03:36:51Z,Tokyo Olympics: Asahi Shimbun newspaper says J...,https://www.theguardian.com/world/2021/may/26/...,https://content.guardianapis.com/world/2021/ma...,False,pillar/news,News


Many APIs, including this one, will split a large result set into multiple pages.  This example gets all pages of results by making multiple requests to the API:

In [6]:
# Code to get all pages of data for a query

# Get the first page
requestUrl = getRequestUrl(pageNo=1)
numPages, results = getApiData(requestUrl)
print("Found {} results".format(len(results)))

# Load into a Pandas dataframe
df = pd.DataFrame(results)

# Get the rest of the pages
for page in range(2, numPages+1):
    requestUrl = getRequestUrl(pageNo=page)
    numpPages, results = getApiData(requestUrl)
    print("Found {} results".format(len(results)))
    
    # Merge into the pandas dataframe
    df = pd.concat([df, pd.DataFrame(results)])

df.head()

Response: status ok, pages 4, pageSize 50, total 159
Found 50 results
Response: status ok, pages 4, pageSize 50, total 159
Found 50 results
Response: status ok, pages 4, pageSize 50, total 159
Found 50 results
Response: status ok, pages 4, pageSize 50, total 159
Found 9 results


,id,type,sectionId,sectionName,webPublicationDate,webTitle,webUrl,apiUrl,isHosted,pillarId,pillarName
0,world/2021/jun/04/tokyo-cornered-into-going-ah...,article,world,World news,2021-06-04T10:18:30Z,"Tokyo cornered into going ahead with Games, sa...",https://www.theguardian.com/world/2021/jun/04/...,https://content.guardianapis.com/world/2021/ju...,False,pillar/news,News
1,sport/2021/jun/07/sally-fitzgibbons-continues-...,article,sport,Sport,2021-06-07T05:26:47Z,Sally Fitzgibbons continues hot pre-Olympic fo...,https://www.theguardian.com/sport/2021/jun/07/...,https://content.guardianapis.com/sport/2021/ju...,False,pillar/sport,Sport
2,sport/2021/jun/02/olympics-over-a-barrel-grim-...,article,sport,Sport,2021-06-02T15:15:47Z,IOC holds Tokyo as Olympic hostage to fortune ...,https://www.theguardian.com/sport/2021/jun/02/...,https://content.guardianapis.com/sport/2021/ju...,False,pillar/sport,Sport
3,world/2021/jun/03/olympics-will-go-ahead-says-...,article,world,World news,2021-06-03T05:11:03Z,Olympics chief says Games will proceed after C...,https://www.theguardian.com/world/2021/jun/03/...,https://content.guardianapis.com/world/2021/ju...,False,pillar/news,News
4,world/2021/may/26/tokyo-olympics-asahi-shimbun...,article,world,World news,2021-05-26T03:36:51Z,Tokyo Olympics: Asahi Shimbun newspaper says J...,https://www.theguardian.com/world/2021/may/26/...,https://content.guardianapis.com/world/2021/ma...,False,pillar/news,News
